In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
import nltk
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import f1_score

from utils.utils import cleanText
from utils.utils import tokenizerGenerate
from utils.utils import tokenize
from utils.utils import embeddingMatrixGenerate

Using TensorFlow backend.


In [2]:
train  = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
upload = pd.read_csv("sample_upload.csv")

In [3]:
train_class0=train[train['class']==0]
train_class1=train[train['class']==1]
train_class2=train[train['class']==2]


train_class1_1=train_class1.iloc[0:2800,:]
train_class1_2=train_class1.iloc[2801:5600,:]
train_class1_3=train_class1.iloc[5601:8400,:]
train_class1_4=train_class1.iloc[8401:11491,:]

train1=pd.concat([train_class0,train_class2,train_class1_1],axis=0)
train2=pd.concat([train_class0,train_class2,train_class1_2],axis=0)
train3=pd.concat([train_class0,train_class2,train_class1_3],axis=0)
train4=pd.concat([train_class0,train_class2,train_class1_4],axis=0)

In [4]:
SEQUENCE_LENGTH = 25
EMBEDDING_DIM = 100

text = train['tweet'].tolist()
clean_text = cleanText(text)
tokenizer = tokenizerGenerate(clean_text)

embeddingMatrix = embeddingMatrixGenerate(EMBEDDING_DIM, tokenizer.word_index)

In [10]:
SEQUENCE_LENGTH = 25

#test
text = test['tweet'].tolist()
clean_text = cleanText(text)
token_list = tokenize(clean_text, tokenizer, SEQUENCE_LENGTH)
LENGTH = len(test)
test_text_vector = np.zeros((LENGTH, EMBEDDING_DIM))
for i in range(LENGTH):
    tmp = np.zeros(EMBEDDING_DIM)
    for j in range(SEQUENCE_LENGTH):
        tmp+=embeddingMatrix[token_list[i][j]]
    test_text_vector[i] = tmp
    
#train
text = train['tweet'].tolist()
clean_text = cleanText(text)
token_list = tokenize(clean_text, tokenizer, SEQUENCE_LENGTH)
LENGTH = len(train)
train_text_vector = np.zeros((LENGTH, EMBEDDING_DIM))
for i in range(LENGTH):
    tmp = np.zeros(EMBEDDING_DIM)
    for j in range(SEQUENCE_LENGTH):
        tmp+=embeddingMatrix[token_list[i][j]]
    train_text_vector[i] = tmp

In [11]:
#train1
text = train1['tweet'].tolist()
clean_text = cleanText(text)
token_list = tokenize(clean_text, tokenizer, SEQUENCE_LENGTH)
LENGTH = len(train1)
text_vector = np.zeros((LENGTH , EMBEDDING_DIM))
for i in range(LENGTH ):
    tmp = np.zeros(EMBEDDING_DIM)
    for j in range(SEQUENCE_LENGTH):
        tmp+=embeddingMatrix[token_list[i][j]]
    text_vector[i] = tmp
    
Model = LogisticRegression(C=250).fit(text_vector,train1['class'])
predictions1 = Model.predict_proba(test_text_vector)
predictions1_train = Model.predict_proba(train_text_vector)

#train2
text = train2['tweet'].tolist()
clean_text = cleanText(text)
token_list = tokenize(clean_text, tokenizer, SEQUENCE_LENGTH)
LENGTH = len(train2)
text_vector = np.zeros((LENGTH , EMBEDDING_DIM))
for i in range(LENGTH ):
    tmp = np.zeros(EMBEDDING_DIM)
    for j in range(SEQUENCE_LENGTH):
        tmp+=embeddingMatrix[token_list[i][j]]
    text_vector[i] = tmp
    
Model = LogisticRegression(C=250).fit(text_vector,train2['class'])
predictions2 = Model.predict_proba(test_text_vector)
predictions2_train = Model.predict_proba(train_text_vector)

#train3
text = train3['tweet'].tolist()
clean_text = cleanText(text)
token_list = tokenize(clean_text, tokenizer, SEQUENCE_LENGTH)
LENGTH = len(train3)
text_vector = np.zeros((LENGTH , EMBEDDING_DIM))
for i in range(LENGTH ):
    tmp = np.zeros(EMBEDDING_DIM)
    for j in range(SEQUENCE_LENGTH):
        tmp+=embeddingMatrix[token_list[i][j]]
    text_vector[i] = tmp
    
Model = LogisticRegression(C=250).fit(text_vector,train3['class'])
predictions3 = Model.predict_proba(test_text_vector)
predictions3_train = Model.predict_proba(train_text_vector)

#train4
text = train4['tweet'].tolist()
clean_text = cleanText(text)
token_list = tokenize(clean_text, tokenizer, SEQUENCE_LENGTH)
LENGTH = len(train4)
text_vector = np.zeros((LENGTH , EMBEDDING_DIM))
for i in range(LENGTH ):
    tmp = np.zeros(EMBEDDING_DIM)
    for j in range(SEQUENCE_LENGTH):
        tmp+=embeddingMatrix[token_list[i][j]]
    text_vector[i] = tmp
    
Model = LogisticRegression(C=250).fit(text_vector,train4['class'])
predictions4 = Model.predict_proba(test_text_vector)
predictions4_train = Model.predict_proba(train_text_vector)

/home/pd2/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/pd2/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/pd2/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/pd2/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/pd2/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: 

In [12]:
predictions_analysis_train=np.concatenate((predictions1_train,predictions2_train,predictions3_train,predictions4_train),axis=1)

predictions_analysis=np.concatenate((predictions1,predictions2,predictions3,predictions4),axis=1)

from sklearn.model_selection import GridSearchCV
parameters = {
    'penalty': ('l1', 'l2'),
    'C': (0.1, 1, 10)
}
LR = LogisticRegression()
FinalModel=GridSearchCV(LR, parameters, cv=5)
FinalModel.fit(predictions_analysis_train,train['class'])
print(FinalModel.best_score_)
print(FinalModel.best_params_)

/home/pd2/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/pd2/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/pd2/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/pd2/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/pd2/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: 

/home/pd2/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/pd2/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/pd2/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/pd2/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/pd2/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: 

0.8793462909408837
{'C': 0.1, 'penalty': 'l1'}


In [13]:
predictions = FinalModel.predict(predictions_analysis)
submission = pd.DataFrame({'id':test['id'],'class':predictions})
file_name = 'test_predictions.csv'
submission.to_csv(file_name,index=False)

In [15]:
check = pd.read_csv('test_predictions.csv')
check[:20]

,id,class
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
5,5,1
6,6,1
7,7,1
8,8,1
9,9,1
